In [13]:
import pandas as pd
import pymysql

In [15]:
myconnection = pymysql.connect(
                host = "127.0.0.1",
                user="root",
                passwd="Aadhu#1207",
                database = "Nutrition")

cursor_N = myconnection.cursor()

In [39]:
## Defining a function for fetching query results ####
def query_result(query):
    cursor_N.execute(query)
    result = cursor_N.fetchall()
    columns = [desc[0] for desc in cursor_N.description]
    data = pd.DataFrame(result,columns=columns)
    return data

In [ ]:
### Setting option to display 500 rows ###
pd.set_option('display.max_rows',500)

**Queries on Obesity Table**

In [47]:
## Query 1 ##
## Top 5 regions with the highest average obesity levels in the most recent year(2022) ##
Q1 = query_result("""
SELECT region AS REGION,ROUND(AVG(mean_estimate),2) AS AVG_ESTIMATE
FROM obesity 
WHERE year ='2022' 
GROUP BY region 
ORDER BY AVG(mean_estimate) DESC LIMIT 5
""")
Q1

,REGION,AVG_ESTIMATE
0,Western Pacific,22.61
1,Americas,21.22
2,Eastern Mediterranean,18.04
3,High Income,16.95
4,Upper Middle Income,13.63


In [49]:
## Query 2 ##
## Top 5 countries with highest obesity estimates ##
Q2 = query_result("""
SELECT Country AS COUNTRY, ROUND(AVG(Mean_Estimate),2) AS AVG_ESTIMATE 
FROM obesity 
GROUP BY Country
ORDER BY AVG_Estimate DESC limit 5
""")
Q2

,COUNTRY,AVG_Estimate
0,Niue,40.66
1,Cook Islands,39.92
2,Nauru,38.51
3,Tokelau,37.96
4,American Samoa,36.89


In [51]:
## Query 3 ##
## Obesity trend in India over the years(Mean_estimate) ##
Q3 = query_result("""
SELECT year AS YEAR,ROUND(AVG(Mean_Estimate),2) AS AVG_ESTIMATE
FROM obesity 
WHERE country='India'
GROUP BY year 
ORDER BY year
""")
Q3

,YEAR,AVG_ESTIMATE
0,2012,2.47
1,2013,2.63
2,2014,2.80
3,2015,2.97
4,2016,3.16
5,2017,3.36
6,2018,3.58
7,2019,3.81
8,2020,4.05
9,2021,4.31


In [55]:
## Query 4 ##
## Average obesity by gender ##
Q4 = query_result("""
SELECT gender AS GENDER,ROUND(AVG(mean_estimate),2) AS AVG_ESTIMATE
FROM obesity 
GROUP BY gender
""")
Q4

,GENDER,AVG_ESTIMATE
0,MALE,12.78
1,FEMALE,12.72
2,BOTH,12.78


In [63]:
## Query 5 ##
## Country count by obesity level category and age group ##
Q5 = query_result("""
SELECT obesity_level,age_group,COUNT(distinct country) AS COUNTRY_COUNT 
FROM obesity 
GROUP BY obesity_level,age_group
""")
Q5

,obesity_level,age_group,COUNTRY_COUNT
0,High,ADULT,94
1,High,CHILD,11
2,Low,ADULT,176
3,Low,CHILD,210
4,Moderate,ADULT,99
5,Moderate,CHILD,26


In [67]:
## Query 6 ##
## Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width) ##
Q6 = query_result("""
(SELECT country,AVG(ci_width) AS Avg_CI_Width,'LEAST RELIABLE' AS Consistency
FROM obesity
GROUP BY Country
ORDER BY Avg_CI_Width DESC LIMIT 5)
UNION ALL
(SELECT Country,AVG(CI_Width) AS Avg_CI_Width,'MOST CONSISTENT'
FROM obesity
GROUP BY Country
ORDER BY Avg_CI_Width ASC LIMIT 5)
""")
Q6

,country,Avg_CI_Width,Consistency
0,Bermuda,24.593418,LEAST RELIABLE
1,American Samoa,23.113530,LEAST RELIABLE
2,Antigua and Barbuda,22.016214,LEAST RELIABLE
3,Puerto Rico,21.698679,LEAST RELIABLE
4,Grenada,21.458488,LEAST RELIABLE
5,Global,0.708952,MOST CONSISTENT
6,Low & Middle Income,0.861059,MOST CONSISTENT
7,South-East Asia Region,1.017775,MOST CONSISTENT
8,India,1.140133,MOST CONSISTENT
9,Japan,1.155902,MOST CONSISTENT


In [69]:
## Query 7 ##
## Average obesity by age group ##
Q7 = query_result("""
SELECT age_group,AVG(mean_estimate) AS AVG_Estimate
FROM obesity 
GROUP BY age_group
""")
Q7

,age_group,AVG_Estimate
0,ADULT,21.833224
1,CHILD,9.736980


In [71]:
## Query 8 ##
## Top 10 Countries with consistent low obesity (low average + low CI)over the years ##
Q8 = query_result("""
SELECT Country,AVG(Mean_Estimate) AS Avg_Obesity,AVG(CI_Width) AS Avg_CI,(AVG(Mean_Estimate) + AVG(CI_Width)) AS Combined_Estimate
FROM obesity
GROUP BY Country
ORDER BY Combined_Estimate ASC LIMIT 10
""")
Q8

,Country,Avg_Obesity,Avg_CI,Combined_Estimate
0,Timor-Leste,0.994067,1.273569,2.267636
1,Ethiopia,1.491995,1.589414,3.081409
2,Burkina Faso,1.721496,1.531514,3.253010
3,Bangladesh,1.976673,1.663507,3.640179
4,Rwanda,1.767194,2.360578,4.127772
5,India,3.429781,1.140133,4.569914
6,Malawi,2.391324,2.632713,5.024037
7,Japan,4.000332,1.155902,5.156235
8,South-East Asia Region,4.236563,1.017775,5.254338
9,Mozambique,2.772244,2.764630,5.536874


In [73]:
## Query 9 ##
## Countries where female obesity exceeds male by large margin (same year) ##
Q9 = query_result("""
SELECT O1.country,O1.year,O1.mean_estimate AS Female_Obesity,
O2.mean_estimate AS Male_Obesity,(O1.mean_estimate - O2.mean_estimate) AS Margin_Difference
FROM obesity O1
JOIN obesity O2
ON O1.country = O2.country AND O1.year = O2.year AND O1.age_group = O2.age_group
WHERE O1.gender = 'FEMALE' AND O2.gender = 'MALE'
AND (O1.mean_estimate - O2.mean_estimate) > 25 
ORDER BY country,year
""")
Q9

,country,year,Female_Obesity,Male_Obesity,Margin_Difference
0,Dominica,2016,39.6822,14.56480,25.117342
1,Dominica,2017,40.5221,15.08610,25.436009
2,Dominica,2018,41.3652,15.62680,25.738411
3,Dominica,2019,42.1942,16.18370,26.010569
4,Dominica,2020,43.0205,16.76600,26.254450
5,Dominica,2021,43.8493,17.37880,26.470419
6,Dominica,2022,44.6827,18.02640,26.656342
7,Egypt,2020,54.1999,28.98730,25.212658
8,Egypt,2021,55.1129,29.52630,25.586552
9,Egypt,2022,56.0083,30.06740,25.940889


In [77]:
## Query 10 ##
## Global average obesity percentage per year ##
Q10 = query_result("""
SELECT Year,ROUND(AVG(Mean_Estimate),2) AS AVG_Obesity_Percentage_Per_Year
FROM obesity 
GROUP BY Year 
ORDER BY Year
""")
Q10

,Year,AVG_Obesity_Percentage_Per_Year
0,2012,11.06
1,2013,11.36
2,2014,11.67
3,2015,11.99
4,2016,12.33
5,2017,12.69
6,2018,13.06
7,2019,13.44
8,2020,13.84
9,2021,14.25


**Queries on Malnutrition Table**

In [82]:
## Query 1 ##
## Avg. malnutrition by age group ##
Q1 = query_result("""
SELECT Age_Group,ROUND(AVG(Mean_Estimate),2) AS AVG_Malnutrition
FROM Malnutrition
GROUP BY Age_Group
""")
Q1

,Age_Group,AVG_Malnutrition
0,ADULT,5.94
1,CHILD,5.16


In [84]:
## Query 2 ##
## Top 5 countries with highest malnutrition(mean_estimate) ##
Q2 = query_result("""
SELECT Country,	ROUND(AVG(Mean_Estimate),2) AS Malnutrition_Percentage
FROM malnutrition 
GROUP BY Country
ORDER BY Malnutrition_Percentage DESC LIMIT 5
""")
Q2

,Country,Malnutrition_Percentage
0,India,22.14
1,South-East Asia Region,19.24
2,Bangladesh,18.25
3,Sri Lanka,17.87
4,Eritrea,17.47


In [86]:
## Query 3 ##
## Malnutrition trend in African region over the years ##
Q3 = query_result("""
SELECT year,ROUND(AVG(Mean_Estimate),2) AS AVG_Estimate
FROM malnutrition
WHERE Region='Africa'
GROUP BY Year 
ORDER BY Year
""")
Q3

,year,AVG_Estimate
0,2012,8.87
1,2013,8.77
2,2014,8.69
3,2015,8.61
4,2016,8.55
5,2017,8.49
6,2018,8.44
7,2019,8.40
8,2020,8.36
9,2021,8.33


In [88]:
## Query 4 ##
## Gender-based average malnutrition ##
Q4 = query_result("""
SELECT Gender,ROUND(AVG(Mean_Estimate),2) As AVG_Malnutrition_Percentage
FROM malnutrition
GROUP BY Gender
""")
Q4

,Gender,AVG_Malnutrition_Percentage
0,MALE,6.03
1,FEMALE,4.68
2,BOTH,5.36


In [92]:
## Query 5 ##
## Malnutrition level-wise (average CI_Width by age group) ##
Q5 = query_result("""
SELECT Age_Group,Malnutrition_Level,ROUND(AVG(CI_Width),2) AS AVG_CI_Width
FROM malnutrition
GROUP BY Age_Group,malnutrition_level
""")
Q5

,Age_Group,Malnutrition_Level,AVG_CI_Width
0,ADULT,Low,2.64
1,ADULT,Moderate,7.64
2,ADULT,High,9.25
3,CHILD,Low,4.44
4,CHILD,Moderate,9.89
5,CHILD,High,7.88


In [94]:
## Query 6 ##
## Yearly malnutrition change in specific countries(India, Nigeria, Brazil) ##
Q6 = query_result("""
SELECT Country,Year,AVG(Mean_Estimate) AS AVG_Malnutrition_Percentage
FROM malnutrition
WHERE Country IN ('India','Nigeria','Brazil')
GROUP BY Year,Country
ORDER BY Country,Year
""")
Q6

,Country,Year,AVG_Malnutrition_Percentage
0,Brazil,2012,2.838653
1,Brazil,2013,2.781387
2,Brazil,2014,2.735034
3,Brazil,2015,2.698948
4,Brazil,2016,2.672242
5,Brazil,2017,2.653567
6,Brazil,2018,2.641574
7,Brazil,2019,2.634596
8,Brazil,2020,2.629412
9,Brazil,2021,2.626638


In [96]:
## Query 7 ##
## Regions with lowest malnutrition averages ##
Q7 = query_result("""
SELECT Region,AVG(Mean_Estimate) AS AVG_Malnutrition_Percentage
FROM malnutrition
GROUP BY Region 
ORDER BY AVG_Malnutrition_Percentage LIMIT 5
""")
Q7


,Region,AVG_Malnutrition_Percentage
0,High Income,2.322331
1,Europe,2.429001
2,Americas,3.144475
3,Western Pacific,3.287015
4,Upper Middle Income,4.475993


In [104]:
## Query 8 ##
## Countries with increasing malnutrition ##
Q8 = query_result("""
WITH avg_per_year AS (
    SELECT country,year,AVG(mean_estimate) AS avg_estimate
    FROM malnutrition
    GROUP BY country, year
	),
yearly_comparison AS (
    SELECT a.country,a.year AS current_year,a.avg_estimate AS current_estimate,b.year AS previous_year,b.avg_estimate AS previous_estimate
    FROM avg_per_year a
    JOIN avg_per_year b
    ON a.country = b.country AND a.year = b.year+1
    WHERE a.avg_estimate <= b.avg_estimate
)
SELECT DISTINCT country
FROM avg_per_year
WHERE country NOT IN (SELECT country FROM yearly_comparison)
""")
Q8

,country
0,Argentina
1,Mongolia
2,United Kingdom
3,Honduras
4,Cook Islands
5,Germany
6,"Venezuela, Bolivarian Republic of"
7,Cyprus
8,Tonga
9,Italy


In [106]:
## Query 9 ##
## Min/Max malnutrition levels year-wise comparison ##
Q9 = query_result("""
SELECT MIN(Mean_Estimate) ,MAX(Mean_Estimate) ,Year
FROM malnutrition
GROUP BY Year
ORDER BY Year
""")
Q9

,MIN(Mean_Estimate),MAX(Mean_Estimate),Year
0,0.17531,35.8941,2012
1,0.17642,35.6944,2013
2,0.17777,35.4982,2014
3,0.17844,35.3130,2015
4,0.17880,35.1477,2016
5,0.17929,34.9979,2017
6,0.17980,34.8562,2018
7,0.18043,34.7160,2019
8,0.18199,34.5784,2020
9,0.18472,34.4292,2021


In [108]:
## Query 10 ##
## High CI_Width flags for monitoring(CI_width > 5) ##
Q10 = query_result("""
SELECT country,ROUND(AVG(ci_width),2) as AVG_Width
FROM malnutrition 
GROUP BY country
HAVING AVG_Width > '5'
ORDER BY AVG_WIDTH DESC
""")
Q10

,country,AVG_Width
0,"Korea, Democratic People's Republic of",16.31
1,Djibouti,14.66
2,Equatorial Guinea,14.58
3,Eritrea,14.32
4,Central African Republic,14.16
5,Somalia,13.68
6,South Sudan,13.10
7,Chad,13.06
8,Senegal,12.56
9,Sudan,12.44


**Combined Queries**

In [115]:
## Query1 ##
## Obesity vs malnutrition comparison by country(any 5 countries) ##
Q1 = query_result("""
SELECT obs.country,ROUND(AVG(obs.mean_estimate),2) AS AVG_Obesity , ROUND(AVG(mal.mean_estimate),2) AS AVG_Malnutrition
FROM obesity obs,malnutrition mal
WHERE obs.country = mal.country 
AND obs.country IN ('India','Somalia','United States','Oman','Japan')
GROUP BY obs.country
""")

Q1

,country,AVG_Obesity,AVG_Malnutrition
0,India,3.43,22.14
1,Japan,4.00,3.84
2,Somalia,4.68,10.03
3,Oman,18.30,6.64
4,United States,25.24,1.30


In [119]:
## Query2 ##
## Gender-based disparity in both obesity and malnutrition ##
Q2 = query_result("""
SELECT obs.year,obs.gender,ROUND(AVG(obs.mean_estimate),2) AS AVG_Obesity,ROUND(AVG(mal.mean_estimate),2) AS AVG_Malnutrition
FROM obesity obs,malnutrition mal
WHERE obs.gender = mal.gender
AND obs.year = mal.year
AND obs.country = mal.country
GROUP BY obs.year,obs.gender
ORDER BY obs.year
""")
Q2

,year,gender,AVG_Obesity,AVG_Malnutrition
0,2012,BOTH,11.08,5.57
1,2012,FEMALE,11.09,4.82
2,2012,MALE,11.02,6.31
3,2013,BOTH,11.38,5.52
4,2013,FEMALE,11.38,4.78
5,2013,MALE,11.32,6.23
6,2014,BOTH,11.69,5.46
7,2014,FEMALE,11.68,4.75
8,2014,MALE,11.64,6.16
9,2015,BOTH,12.02,5.42


In [121]:
## Query3 ##
## Region-wise avg estimates side-by-side(Africa and America) ##
Q3 = query_result("""
SELECT obs.region,ROUND(AVG(obs.mean_estimate),2) AS AVG_Obesity,ROUND(AVG(mal.mean_estimate),2) AS AVG_Malnutrition
FROM obesity obs,malnutrition mal
WHERE obs.region = mal.region
AND obs.country = mal.country
AND obs.region IN ('Africa','Americas')
GROUP BY obs.region
""")
Q3
 

,region,AVG_Obesity,AVG_Malnutrition
0,Americas,18.43,3.14
1,Africa,5.40,8.53


In [127]:
## Query4 ##
## Countries with obesity up & malnutrition down ##
Q4 = query_result("""
SELECT obs.country ,ROUND(AVG(obs.mean_estimate),2) AS AVG_Obesity,ROUND(AVG(mal.mean_estimate),2) AS AVG_Malnutrition
FROM obesity obs JOIN malnutrition mal
ON obs.country = mal.country
AND obs.region = mal.region
AND obs.year = mal.year
GROUP BY obs.country
HAVING AVG_Obesity > '25'
AND AVG_Malnutrition < '5'
ORDER BY AVG_Obesity DESC
""")
Q4

,country,AVG_Obesity,AVG_Malnutrition
0,Niue,40.66,0.73
1,Cook Islands,39.92,0.75
2,Nauru,38.51,0.69
3,Tokelau,37.96,0.90
4,American Samoa,36.89,0.73
5,Tonga,36.61,0.39
6,Tuvalu,33.97,1.41
7,Bahamas,29.40,3.47
8,French Polynesia,29.20,1.47
9,Samoa,29.06,0.85


In [129]:
## Query 5 ##
## Age-wise trend analysis ##
Q5 = query_result("""
SELECT obs.year,obs.age_group,
ROUND(AVG(obs.mean_estimate),2) AS AVG_Obesity,ROUND(AVG(mal.mean_estimate),2) AS AVG_Malnutrition
FROM obesity obs JOIN malnutrition mal
ON obs.age_group = mal.age_group
AND obs.year = mal.year
AND obs.country = mal.country
GROUP BY obs.age_group,obs.year
ORDER BY obs.year
""")
Q5

,year,age_group,AVG_Obesity,AVG_Malnutrition
0,2012,ADULT,19.52,6.44
1,2012,CHILD,8.24,5.27
2,2013,ADULT,19.95,6.32
3,2013,CHILD,8.50,5.24
4,2014,ADULT,20.39,6.21
5,2014,CHILD,8.76,5.21
6,2015,ADULT,20.85,6.10
7,2015,CHILD,9.04,5.18
8,2016,ADULT,21.32,6.00
9,2016,CHILD,9.34,5.16


In [133]:
## Finally closing the connection ##

cursor_N.close()
myconnection.close()

Error: Already closed